## Chunking and Parallelizing

__Chunking__ is useful for whenever a task can be accomplished one piece of data at a time. Even some applications that may look like they can't be done using chunking can be done if you think about the problem in the right way!

Obvious example: __how would we do a summation using chunking?__

In [ ]:
import pandas as pd
import numpy as np


By feeding the `read_csv` function the `chunksize` argument, it becomes an iterable reader:

In [ ]:
chonk = pd.read_csv("./ex_big.csv",chunksize=10000)

chonk

If we wanted to count the number of 1s, for example, we could do the following:

In [ ]:
s = 0
for i in chonk:
    s += i["0"].sum()

In [ ]:
s

We can also do subsetting this way. Suppose that I wanted to select only cows that are breed "AY." What I'll do is first write an empty dataframe with the column names, then cycle through the columns and writing out only the rows I want.

In [ ]:
file_dir="C:/Users/jhtchns2/Box/ACE 592 SAE - Spring 2023/Class_Examples/3_Numeric/"

# Read in one row, take the columns
cols = pd.read_csv(file_dir+"naab_example.csv",nrows=1).columns

# Write out an empty dataframe with just the column names
pd.DataFrame(columns=cols).to_csv("./out.csv",index=False)

In [ ]:
# Get that chonker running.
chonker = pd.read_csv(file_dir+"naab_example.csv",chunksize=10000)

for i in chonker:
    i[i['breed']=="AY"].to_csv("./out.csv", mode='a', header=False)
    
pd.read_csv("out.csv")

And nothing had to be run in memory!

In [ ]:
import glob

trade_files = glob.glob("P:trade_data/BACI_HS12_V202201/BACI_HS12_Y*.csv")

for F in trade_files:
    chonker = pd.read_csv(F,chunksize=100000)
    
    # Read in one row, take the columns
    cols = pd.read_csv(F,nrows=1).columns

    # Write out an empty dataframe with just the column names
    pd.DataFrame(columns=cols).to_csv(F.split(".csv")[0]+"_m.csv",index=False)
    
    for i in chonker:
        i['v'] = pd.to_numeric(i['v'],errors="coerce")
        i['q'] = pd.to_numeric(i['q'],errors="coerce")
        i.to_csv(F.split(".csv")[0]+"_m.csv", mode='a', header=False)

In [ ]:
import os

for F in trade_files:
    os.replace(F.split(".csv")[0]+"_m.csv",F)

#### One example of chunking that I needed to do for the homework.

Now how might we do chunking to calculate the mean and variance of these two arrays?

In [ ]:
sample = np.random.normal(loc=[4., 20.], scale=[1., 3.5],
                           size=(100000000, 2))

In [ ]:
sample

__Try it!__

### Threading and Parallelization

Within one CPU, you can create multiple __threads__ to do tasks concurrently. Across all CPUs, you can create multiple __processes__ to do tasks in parallel.

We will mostly be talking about __parallelization__ for computations, but threading is more applicable for less CPU intensive tasks.

To parallelize code means to allow your computer cores to run independently but in parallel. This can be helpful when your task needs to be repeated for a list of inputs and collected at the end. __Programs can be parallelized only if each run can be done independently of the other__.

Parallel operations:
- Bootstrapping a statistic.
- Scraping a list of webpages.
- Grid search.


Non-parallel operations:
- Evaluations of an optimization routine.
- Rolling statistics.
- The traveling salesman problem.

__Why?__

#### Types of parallelization
- Synchronous: locks the main program until all processes are finished and returns it in the order it was given.
    - Example: in a program where two cores need to square four numbers, the program stops until the cores finish all the numbers. They return the list in the order that they were given.
- Asynchronous: does not lock the main program and returns it in the order it finishes.
    - Example: in that same program, the program will not stop but the number return in the order that they were finished squaring.

In [ ]:
import multiprocess as mp

def square(x):
    return x*x

import numpy as np
X = np.arange(1,2000000,1)

Non-parallelized

In [ ]:
[x*x for x in X]

Threading

In [ ]:
from multiprocess.pool import ThreadPool

thread_pool = ThreadPool(5)
res_thread = thread_pool.map(square,X)
thread_pool.close()
thread_pool.join()

In [ ]:
res_thread[:10]

Synchronous Parallel

In [ ]:
pool = mp.Pool(5)
res_synchr = pool.map(square,X)
pool.close()
pool.join()

In [ ]:
res_synchr[:10]

Asynchronous Parallel

In [ ]:
pool = mp.Pool(5)
res_async = pool.map_async(square,X)
pool.close()
pool.join()

In [ ]:
res_async.get()[:10]

It is good practice to call `pool.close()` and `pool.join()` in order to close out the processes when you are done.

### What took the least amount of time?

### Threading was the fastest. Why?

- It didn't have to create processes, which needs a new python interpreter each time.
- The major bottleneck was getting data, not the computation.

Another advantage is that it could write to a common array `L`...

This leads to a general principle with threading versus parallel:
- If I/O bound problem, use threading.
    - I/O bound means the most time-consuming part is reading or writing data.
- If CPU bound problem, use parallel.
    - CPU bound means the most time-consuming part is processing the data.
    
Squaring a number is not CPU intensive, and setting up new processes is expensive.

### A basic bootstrap using `multiprocessing`


In [ ]:
sample = np.random.normal(loc=[4., 20.], scale=[20., 40.5],
                           size=(10000000, 2))

In [ ]:
sample

In [ ]:
ixs = np.random.randint(len(sample),size=len(sample))
sample[ixs,:]

In [ ]:
def bootstrap(i):
    import numpy as np
    ixs = np.random.randint(len(sample),size=len(sample))
    df = sample[ixs,:]
    return df.mean(axis=0)

In [ ]:
pool = mp.Pool(4)
res_synchr = pool.map(bootstrap,range(50))
pool.close()
pool.join()

In [ ]:
pool.close()
pool.join()

What happened here?

Unfortunately, the new process it creates does not have any of the things in memory it had before. This demonstrates the difference between __threads__ and __processes__:

- Threads use the same memory space on 1 CPU and are a __subset of a process__:
    - Good if you want threads to share data.
    - Bad if those objects are being modified at the same time, because each thread will __lock__ it until its done.
- Processes carve out their own memory space to use on multiple CPUs. They are the __superset of threads__:
    - Good if computation heavy and needs to edit each object.
    - Bad if the operation is simple, because it takes up a lot of memory.

Threading is best for lightweight but long activities. Processes are better for computation heavy things that need their own carved out memory.

Using threads, we can edit a common object:

In [ ]:
L = []
def square_append(x):
    L.append(x*x)
    return x*x

thread_pool = ThreadPool(5)
res_thread = thread_pool.map(square_append,X)
thread_pool.close()
thread_pool.join()

In [ ]:
L[:10]

So we can use bootstrapping with threads:

In [ ]:
thread_pool = ThreadPool(5)
res_thread = thread_pool.map(bootstrap,range(50))
thread_pool.close()
thread_pool.join()

In [ ]:
np.concatenate(res_thread).reshape(50,2).mean(axis=0)

If we want to use parallel processing, we need to edit the function a little bit.

In [ ]:
import pandas as pd
pd.DataFrame(sample).to_csv("sample.csv",index=False)

In [ ]:
def bootstrap_read(i):
    import numpy as np
    import pandas as pd
    ixs = np.random.randint(10000000,size=10000000)
    df = pd.read_csv("sample.csv").iloc[ixs,:]
    return df.mean(axis=0).values

In [ ]:
pool = mp.Pool(4)
res_synchr = pool.map(bootstrap_read,range(50))
pool.close()
pool.join()

In [ ]:
Res = np.concatenate(res_synchr)

Res.reshape(50,2).mean(axis=0),\
Res.reshape(50,2).std(axis=0)

Lets benchmark against the sequential way

In [ ]:
def thread_bootstrap(reps,no_threads=4):
    pool = ThreadPool(no_threads)
    res_synchr = pool.map(bootstrap,range(reps))
    pool.close()
    pool.join()

    res = np.concatenate(res_synchr).reshape(reps,2)
    return res.mean(axis=0),res.std(axis=0)

def parallel_bootstrap(reps,no_workers=4):
    pool = mp.Pool(no_workers)
    res_synchr = pool.map_async(bootstrap_read,range(reps))
    pool.close()
    pool.join()

    res = np.concatenate(res_synchr.get()).reshape(reps,2)
    return res.mean(axis=0),res.std(axis=0)
    
def sequential_bootstrap(reps):
    res = []
    for i in range(reps):
        res += [bootstrap(i)]
    res = np.concatenate(res).reshape(reps,2)
    return res.mean(axis=0),res.std(axis=0)


In [ ]:
%%time 
thread_bootstrap(50)

In [ ]:
%%time 
parallel_bootstrap(50)

In [ ]:
%%time 
sequential_bootstrap(50)

In [ ]:
%%time 
thread_bootstrap(500)

In [ ]:
%%time
parallel_bootstrap(500)

In [ ]:
%%time
sequential_bootstrap(500)

Let's look at how the speed gap changes with B, the number of bootstrap replications:

In [ ]:
import time

def time_bootstrap(func,B=10):
    start = time.time()
    
    func(B)
    
    end = time.time()
    return end-start

In [ ]:
seq_boot = [time_bootstrap(sequential_bootstrap,B=x) for x in np.arange(10,2011,100)]
thr_boot = [time_bootstrap(thread_bootstrap,B=x) for x in np.arange(10,2011,100)]
par_boot = [time_bootstrap(parallel_bootstrap,B=x) for x in np.arange(10,2011,100)]

In [ ]:
boot_res = pd.DataFrame([seq_boot,thr_boot,par_boot],\
                         index=["Sequential","Threading","Parallel"],\
                         columns = np.arange(10,2011,100)).T

boot_res.to_csv("bootstrap_bench.csv")

In [ ]:
import matplotlib.pyplot as plt
boot_res = pd.read_csv("bootstrap_bench.csv").set_index("Unnamed: 0")
boot_res.plot()
plt.xlabel("Replications")
plt.ylabel("Time in Seconds")

In [ ]:
import matplotlib.pyplot as plt
boot_res = pd.read_csv("bootstrap_bench.csv").set_index("Unnamed: 0")
boot_res.iloc[:,:2].plot()
plt.xlabel("Replications")
plt.ylabel("Time in Seconds")

Threading increases in time at a much slower rate than sequential.

In this case, parallel was not faster. Why?


#### This is not a CPU bound task, therefore there is no advantage to parallel.

It takes far more time to initialize the process than it does to run it (it's a very simple calculation). It takes quite a bit of time to reallocate memory.

There is some small overhead to allocate workers, hence why at the lowest value sequential beats parallel.


Were the computation to be CPU bound, doing a simple parallel operation in this case would be much better.

However, the above seems to be heavily dependent on your computer.


There will also be slight speed improvements if there are more workers allocated:

In [ ]:
%%time 
parallel_bootstrap(50,no_workers=8)

In [ ]:
%%time
parallel_bootstrap(50,no_workers=4)

In general, a good rule of thumb is to have as many workers as you CPUs to use.

My computer has:

In [ ]:
mp.cpu_count()

You can use more than your CPU count provided that they are not tasks that require a lot of memory or CPUs for each one. When you exceed your CPU count you split one cpu into multiple tasks.

In this case we are reading in data, which is likely to be very memory intensive. Not in general a good idea to spawn too many of them in this case.

__Note:__ If you don't close the pool you are using, you risk creating too many instances and running out of memory.

In [ ]:
%%time
parallel_bootstrap(100,no_workers=20)

Notice that with too many workers you actually lose speed improvments

In [ ]:
l = []
for i in range(2,20):
    start = time.time()
    parallel_bootstrap(500,no_workers=i)
    proc_time = time.time() - start
    l+= [proc_time]

In [ ]:
pd.Series(l).to_csv("worker_bench.csv",index=False)

In [ ]:
import matplotlib.pyplot as plt

l = pd.read_csv("worker_bench.csv")["0"]

plt.plot(list(range(2,20)),l,label="Parallel")
plt.xlabel("Number of Workers")
plt.ylabel("Time to do 500 reps")
plt.axvline(8,color='black',ls='--',label="Number of Cores")
plt.axhline(24.10096502304077,color="C3",label="Sequential")
plt.ylim(5,30)
plt.legend()

Depending on the run, more workers does not always equal good performance. In general, always start with allocating 1 core to 1 worker.

For most problems, simply setting workers equal to your CPUs is a massive speed improvement.